In [1]:
# sandbox notebook for experimenting with nflgame package
# https://github.com/BurntSushi/nflgame
# api documentation at http://pdoc.burntsushi.net/nflgame
# install with "pip install nflgame"

import nflgame as ng
import numpy as np

In [ ]:
ng.standard_team('carolina panthers') #returns standard abbreviation for team name

In [4]:
for player in ng.find('Cam Newton'): # ng.find() returns a LIST of "player" objects
    
    print player.birthdate
    print player.gsis_id
    print player.team
    
    for play in player.plays(2015, week=1):
        print play.players, play.data['time']

5/11/1989
00-0027939
CAR
[C.Newton, T.Ginn] 13:36
[C.Newton, P.Posluszny] 13:31
[E.Dickson, C.Newton, P.Posluszny] 12:49
[C.Newton, C.Brown, T.Smith] 12:14
[C.Newton, J.Cotchery] 10:07
[C.Newton, T.Ginn] 10:04
[G.Olsen, C.Newton, S.Brown] 09:59
[C.Clemons, C.Newton] 08:25
[E.Dickson, C.Newton, P.Posluszny] 07:58
[C.Newton, D.Funchess] 05:40
[C.Newton, T.Ginn] 05:35
[C.Newton, J.Odrick] 00:19
[R.Miller, C.Newton] 15:00
[J.Stewart, C.Newton, D.Skuta] 09:13
[C.Newton, A.Colvin, D.Funchess] 08:55
[C.Newton, P.Posluszny, G.Olsen] 08:22
[C.Newton, D.Skuta] 07:19
[C.Newton, T.Ginn] 06:35
[C.Newton, J.Cotchery] 06:10
[J.Stewart, J.Cyprien, C.Newton] 05:21
[C.Newton, P.Posluszny, F.Whittaker] 04:47
[C.Newton, C.Brown, D.Gratz] 04:10
[C.Newton, A.Colvin, M.Tolbert, P.Posluszny] 03:07
[R.Davis, C.Newton] 02:25
[C.Newton, D.Skuta] 02:00
[C.Newton, J.Cotchery] 01:56
[C.Newton] 00:14
[C.Newton, T.Ginn] 14:30
[D.House, C.Newton, T.Ginn] 14:23
[C.Newton, T.Smith] 12:50
[C.Newton, D.Skuta] 12:08
[C.Cle

In [ ]:
gamelist = ng.games(2015, week = range(1,18), home = 'CAR', away = 'CAR', kind = 'REG')
# this is: a list of all games with the carolina panthers in the 2015 regular season

# http://pdoc.burntsushi.net/nflgame#nflgame.games says:
# As a special case, if the home and away teams are set to the same team, then all games where that team played are returned.
# 
# The kind parameter specifies whether to fetch preseason, regular season or postseason games. Valid values are PRE, REG and POST.
# 
# The week parameter is relative to the value of the kind parameter, and may be set to a list of week numbers. 
# In the regular season, the week parameter corresponds to the normal week numbers 1 through 17. 
# Similarly in the preseason, valid week numbers are 1 through 4. In the post season, the week number corresponds to 
# the numerical round of the playoffs. So the wild card round is week 1, the divisional round is week 2, the conference round 
# is week 3 and the Super Bowl is week 4.

In [ ]:
nonegamelist = ng.games(2015, week = None, home = 'CAR', away = 'CAR', kind = 'REG')

In [ ]:
# confirms that both lists are the same; week = None is equivalent to week = range(1,18) (makes sense)
# also this is how you get the game ID# from the game object

gamelistkeys = []
for g in gamelist:
    gamelistkeys += [g.gamekey]
    
gamelistnonekeys = []
for g in nonegamelist:
    gamelistnonekeys += [g.gamekey]
    
gamelistkeys == gamelistnonekeys

In [ ]:
# get a list of all plays in a game
# timestamps appear to be given in time OF THAT QUARTER (so all times are <15:00 because each quarter is 15 min) 
# timestamps are given in DECREASING TIME: beginning of the quarter is 15:00. end of the quarter is 0:00.

playsingame0 = list(gamelist[0].drives.plays())
camnewtongsid = '00-0027939'

for play in playsingame0:
    print play.data['qtr'], play.data['time'], play.drive, play.players


In [ ]:
for d in list(gamelist[0].drives):
    print d.time_start, d.time_end, d.result

In [ ]:
drives = list(gamelist[0].drives)
print drives[0].time_start
print drives[0].time_start.qtr
print drives[0].time_start.clock

# see here: http://pdoc.burntsushi.net/nflgame/game.m.html#nflgame.game.GameClock
# drive timestamps are given as GameClock objects....
# looks like drives[0].time_start.clock will give you...the time as a string....

In [ ]:
# trying to get total playtime for cam newton in one game

playsingame0 = list(gamelist[0].drives.plays())
camnewtongsid = '00-0027939'

camnewtonplays = [play for play in playsingame0 if play.has_player(camnewtongsid)]

starttime = [int(t) for t in camnewtonplays[0].data['time'].split(':')] 
starttime = [camnewtonplays[0].data['qtr'] - 1] + starttime #now starttime is list (qtr -  1, min, sec)
starttime_seconds = np.dot(starttime, [15*60, 60,1]) #starttime in seconds since beginning of game

endtime = [int(t) for t in camnewtonplays[-1].data['time'].split(':')]
endtime = [camnewtonplays[-1].data['qtr'] - 1] + endtime
endtime_seconds = np.dot(endtime, [15*60, 60, 1]) #endtime in seconds since beginning of game

playtimegame0 = endtime_seconds - starttime_seconds
print playtimegame0

In [3]:
# trying to get total playtime for cam newton in entire season

gamelist = ng.games(2015, home = 'CAR', away = 'CAR', kind = 'REG')
camnewtongsid = '00-0027939'
netplaytime = 0

for game in gamelist:
    playsingame = list(game.drives.plays())
    camnewtonplays = [play for play in playsingame if play.has_player(camnewtongsid)]

    starttime = [int(t) for t in camnewtonplays[0].data['time'].split(':')] 
    starttime = [camnewtonplays[0].data['qtr'] - 1] + starttime #now starttime is list (qtr -  1, min, sec)
    starttime_seconds = np.dot(starttime, [15*60, 60,1]) #starttime in seconds since beginning of game

    endtime = [int(t) for t in camnewtonplays[-1].data['time'].split(':')]
    endtime = [camnewtonplays[-1].data['qtr'] - 1] + endtime
    endtime_seconds = np.dot(endtime, [15*60, 60, 1]) #endtime in seconds since beginning of game

    gametime = endtime_seconds - starttime_seconds
    print gametime
    netplaytime += gametime
    
print 'net playtime: ', netplaytime

1890
1805
2467
1922
1973
2418
3139
2063
1974
2514
2022
2083
1285
1980
1902
2351
net playtime:  33788


In [15]:
for i in ng.find('mike vick'):
    print i.name, i.team

Mike Vick PIT
